In [1]:
!pip install tensorflow==2.16.1
!pip install tensorflow-gpu==2.12.0
!pip install opencv-python
!pip install matplotlib

  Using cached tensorflow-2.16.1-cp311-cp311-macosx_10_15_x86_64.whl.metadata (4.1 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached gast-0.5.4-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached h5py-3.11.0-cp311-cp311-macosx_10_9_x86_64.whl.metadata (2.5 kB)
  Using cached libclang-18.1.1-py2.py3-none-macosx_10_9_x86_64.whl.metadata (5.2 kB)
  Using cached ml_dtypes-0.3.2-cp311-cp311-macosx_10_9_universal2.whl.metadata (20 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached termcolor-2.4.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached grpcio-1.64.1-cp311-cp311-macosx_10_9_universal2.whl.metadata (3.3 kB)
  Using cached tensorboard-2.16.2-py3-none-any.whl.metadata (1.6 kB)
  Using cached keras-3.3.3-p

In [1]:
import cv2
import os
import random
import numpy as np
import matplotlib.pyplot as plt


In [2]:
#Import tensorflow dependencies - Functional API
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPool2D, Input, Flatten

2024-06-10 21:11:28.298545: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [4]:
#Joins the base path('data') and subdirectory path together to create a full path.
POS_PATH = os.path.join('data', 'positive')
NEG_PATH = os.path.join('data', 'negative')
ANC_PATH = os.path.join('data', 'anchor')

In [5]:
# Creates directories if they don't exist already
paths = [POS_PATH, NEG_PATH, ANC_PATH]
for path in paths:
  if not os.path.exists(path):
    os.makedirs(path)

In [6]:
# Moves LFW images to the NEG_PATH repository
for directory in os.listdir('lfw'):
  for file in os.listdir(os.path.join('lfw', directory)):
    EX_PATH = os.path.join('lfw', directory, file)
    NEW_PATH = os.path.join(NEG_PATH, file)
    os.replace(EX_PATH, NEW_PATH)



NotADirectoryError: [Errno 20] Not a directory: 'lfw/.DS_Store'

In [7]:
# imports library to generate unique image names
import uuid

In [10]:
# Activates Webcam
cap = cv2.VideoCapture(1)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Resize frame to reduce processing load
    frame = cv2.resize(frame, (640, 480))

    # Cut down frame to 250x250px
    frame = frame[200:200 + 250, 200:200 + 250, :]

    # Collect anchors
    if cv2.waitKey(1) & 0xFF == ord('a'):
        # Creates unique file path
        img_name = os.path.join(ANC_PATH, '{}.jpg'.format(uuid.uuid1()))
        # Writes out anchor image
        cv2.imwrite(img_name, frame)

    # Collect positives
    if cv2.waitKey(1) & 0xFF == ord('p'):
        # Creates unique file path
        img_name = os.path.join(POS_PATH, '{}.jpg'.format(uuid.uuid1()))
        # Writes out positive image
        cv2.imwrite(img_name, frame)

    # Show image on screen
    cv2.imshow('Image Collection', frame)

    # Break loop when q is tapped
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Deactivates Webcam
cap.release()

# Closes the window that shows the image frame
cv2.destroyAllWindows()
